# Hadamard Product


## C Implementation

In [28]:
%%writefile hadamard_c.c
#include <stdio.h>
#include <stdlib.h>
#include <time.h>

void dumpArr(size_t row, size_t col, float arr[row][col])
{
  printf("Array output: \n");
  for (size_t i = 0; i < row; ++i) {
    for (size_t j = 0; j < col; ++j) {
      printf("%.2f ", arr[i][j]);
    }
    printf("\n");
  }
}

void C_hadamard(size_t ARR_SIZE, float z[ARR_SIZE][ARR_SIZE], float x[ARR_SIZE][ARR_SIZE], float y[ARR_SIZE][ARR_SIZE])
{
  for (size_t i = 0; i < ARR_SIZE; ++i) {
    for (size_t j = 0; j < ARR_SIZE; ++j) {
      z[i][j] = x[i][j] * y[i][j];
    }
  }
}

int main()
{
  const size_t ARR_SIZE = 4096;
  size_t NUM_EXEC = 10;

  // https://stackoverflow.com/questions/3911400/how-to-pass-2d-array-matrix-in-a-function-in-c
  // int (*array)[cols] = malloc(rows * cols * sizeof(array[0][0]));
  float (*x)[ARR_SIZE] = malloc(ARR_SIZE * ARR_SIZE * sizeof(x[0][0]));
  float (*y)[ARR_SIZE] = malloc(ARR_SIZE * ARR_SIZE * sizeof(y[0][0]));
  float (*z)[ARR_SIZE] = malloc(ARR_SIZE * ARR_SIZE * sizeof(z[0][0]));

  for (size_t i = 0; i < ARR_SIZE; ++i) {
    for (size_t j = 0; j < ARR_SIZE; ++j) {
      x[i][j] = 1.0f;
      y[i][j] = 2.0f;
    }
  }

  clock_t start, end;
  double elapse, time_taken;
  elapse = 0.0f;
  // fill in cache
  C_hadamard(ARR_SIZE, z, x, y);


  for (size_t i = 0; i < NUM_EXEC; ++i) {
      start = clock();
      C_hadamard(ARR_SIZE, z, x, y);
      end = clock();
      time_taken = (end-start)*1E6/CLOCKS_PER_SEC;
      elapse +=  time_taken;
  }


  size_t err_count = 0;

  for (size_t i = 0; i < ARR_SIZE; ++i) {
    for (size_t j = 0; j < ARR_SIZE; ++j) {
      if( (x[i][j] * y[i][j]) != z[i][j] ) {
        err_count++;
      }
    }
  }

  free(x);
  free(y);
  free(z);
  // dumpArr(ARR_SIZE, ARR_SIZE, z);
  printf("Function in C took an average time of %lf in microseconds\n", elapse/NUM_EXEC);
  printf("Total error count: %lu", err_count);

  return 0;
}


Overwriting hadamard_c.c


In [30]:
%%shell
gcc -Wall -Wextra -pedantic -o hadamard_c hadamard_c.c

In [31]:
%%shell
chmod +X ./hadamard_c

In [32]:
%%shell
./hadamard_c

Function in C took an average time of 49203.200000 in microseconds
Total error count: 0

In [33]:
%%writefile hadamard_cuda.cu
#include <cstddef>
#include <cstdio>
#include <stdlib.h>
#include <stdio.h>


void dump_arr(size_t numCol, size_t numRow, float* Arr)
{
    for (size_t j = 0; j < numCol; ++j) {
      for (size_t i = 0; i < numRow; ++i) {
      printf("%.2f ", Arr[i * numCol + j]);
    }
    printf("\n");
  }
}

void C_Hadamard(size_t numCol, size_t numRow, float* Z, float* X, float* Y)
{
  for (size_t i = 0; i < numRow; ++i) {
    for (size_t j = 0; j < numCol; ++j) {
      Z[i * numCol + j] = X[i * numCol + j] * Y[i * numCol + j];
    }
  }
}

__global__
void cuda_hadamard(size_t numCol, size_t numRow, float* Z, float* X, float* Y)
{
  size_t threadRowID = blockIdx.x * blockDim.x + threadIdx.x;
  size_t threadColId = blockIdx.y * blockDim.y + threadIdx.y;

  // Z[threadColId * numCol + threadRowID] = X[threadColId * numCol + threadRowID] * Y[threadColId * numCol + threadRowID];
  // if (threadRowID < numRow && threadColId < numCol) {
  Z[threadRowID * numCol + threadColId] = X[threadRowID * numCol + threadColId] * Y[threadRowID * numCol + threadColId];
  // }

}



int main()
{
  const size_t ARRAY_DIM = 1024; // 4096;
  const size_t ARRAY_BYTES = ARRAY_DIM * ARRAY_DIM * sizeof(float);
  size_t NUM_EXEC = 30;

  // Array Initialization
  float* X;
  float* Y;
  float* Z;

  // Initialize C implementation of Hadamard Product, reference for error checking
  float* C;
  C = (float*)malloc(ARRAY_DIM * ARRAY_DIM * sizeof(float));

  // Array Malloc
  cudaMallocManaged(&X, ARRAY_BYTES);
  cudaMallocManaged(&Y, ARRAY_BYTES);
  cudaMallocManaged(&Z, ARRAY_BYTES);

  // get gpu ID
  int device = -1;
  cudaGetDevice(&device);

  // Mem advise
  cudaMemAdvise(X, ARRAY_BYTES, cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);
  cudaMemAdvise(X, ARRAY_BYTES, cudaMemAdviseSetReadMostly, cudaCpuDeviceId);
  cudaMemAdvise(Y, ARRAY_BYTES, cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);
  cudaMemAdvise(Y, ARRAY_BYTES, cudaMemAdviseSetReadMostly, cudaCpuDeviceId);

  // "prefetch data" to create CPU page memory
  cudaMemPrefetchAsync(X, ARRAY_BYTES, cudaCpuDeviceId, NULL);
  // "prefetch data" to create CPU page memory
  cudaMemPrefetchAsync(Y, ARRAY_BYTES, cudaCpuDeviceId, NULL);
  // "prefetch data" to create GPU page memory
  cudaMemPrefetchAsync(Z, ARRAY_BYTES, device, NULL);

  // initialize array contents
  for (size_t i = 0; i < ARRAY_DIM; ++i) {
    for (size_t j = 0; j < ARRAY_DIM; ++j) {
      X[ARRAY_DIM * i + j] = 1;
      Y[ARRAY_DIM * i + j] = 1;
    }
  }

  // "Prefetch data" from CPU-GPU
  cudaMemPrefetchAsync(X, ARRAY_BYTES, device, NULL);
  cudaMemPrefetchAsync(Y, ARRAY_BYTES, device, NULL);

  C_Hadamard(ARRAY_DIM, ARRAY_DIM, C, X, Y);

  // setup CUDA kernel
  // https://www.cs.emory.edu/~cheung/Courses/355/Syllabus/94-CUDA/2D-grids.html
  size_t threadDimBlockx = 8;
  size_t threadDimBlocky = 8;

  dim3 blockShape = dim3(threadDimBlockx, threadDimBlocky);
  dim3 gridShape = dim3(ARRAY_DIM/threadDimBlockx, ARRAY_DIM/threadDimBlocky);

  for (size_t i = 0; i < NUM_EXEC; ++i) {
    cuda_hadamard <<< gridShape, blockShape >>> (ARRAY_DIM, ARRAY_DIM, Z, X, Y);
  }

  cudaDeviceSynchronize();

  // "Prefetch data" from GPU-CPU
  cudaMemPrefetchAsync(X, ARRAY_BYTES, cudaCpuDeviceId, NULL);
  cudaMemPrefetchAsync(Y, ARRAY_BYTES, cudaCpuDeviceId, NULL);
  cudaMemPrefetchAsync(Z, ARRAY_BYTES, cudaCpuDeviceId, NULL);


  // dump_arr(ARRAY_DIM, ARRAY_DIM, X);
  // printf("\n\n");
  // dump_arr(ARRAY_DIM, ARRAY_DIM, Z);

  // error checking
  size_t errCount = 0;

  for (size_t i = 0; i < ARRAY_DIM; ++i) {
    for (size_t j = 0; j <ARRAY_DIM; ++j ) {
      if (C[i * ARRAY_DIM + j] != Z[i * ARRAY_DIM + j]) {
        errCount++;
      }
    }
  }

  printf("Total error count: %lu", errCount);

  cudaFree(X);
  cudaFree(Y);
  cudaFree(Z);


  return 0;
}


Overwriting hadamard_cuda.cu


In [35]:
%%shell
nvcc -o hadamard_cuda hadamard_cuda.cu -arch=sm_75

In [36]:
%%shell
nvprof ./hadamard_cuda

==11450== NVPROF is profiling process 11450, command: ./hadamard_cuda
Total error count: 0==11450== Profiling application: ./hadamard_cuda
==11450== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  3.1060ms        30  103.53us  96.960us  104.90us  cuda_hadamard(unsigned long, unsigned long, float*, float*, float*)
      API calls:   96.35%  208.91ms         3  69.637ms  37.393us  208.81ms  cudaMallocManaged
                    1.74%  3.7723ms         8  471.54us  12.358us  1.2170ms  cudaMemPrefetchAsync
                    1.36%  2.9569ms         1  2.9569ms  2.9569ms  2.9569ms  cudaDeviceSynchronize
                    0.30%  641.26us         3  213.75us  171.17us  238.65us  cudaFree
                    0.16%  344.48us        30  11.482us  3.8230us  169.19us  cudaLaunchKernel
                    0.07%  149.81us       114  1.3140us     110ns  59.816us  cuDeviceGetAttribute
                    0.01%  29.894us 

In [37]:
%%writefile hadamard_cuda16x16.cu
#include <cstddef>
#include <cstdio>
#include <stdlib.h>
#include <stdio.h>


void dump_arr(size_t numCol, size_t numRow, float* Arr)
{
    for (size_t j = 0; j < numCol; ++j) {
      for (size_t i = 0; i < numRow; ++i) {
      printf("%.2f ", Arr[i * numCol + j]);
    }
    printf("\n");
  }
}

void C_Hadamard(size_t numCol, size_t numRow, float* Z, float* X, float* Y)
{
  for (size_t i = 0; i < numRow; ++i) {
    for (size_t j = 0; j < numCol; ++j) {
      Z[i * numCol + j] = X[i * numCol + j] * Y[i * numCol + j];
    }
  }
}

__global__
void cuda_hadamard(size_t numCol, size_t numRow, float* Z, float* X, float* Y)
{
  size_t threadRowID = blockIdx.x * blockDim.x + threadIdx.x;
  size_t threadColId = blockIdx.y * blockDim.y + threadIdx.y;

  // Z[threadColId * numCol + threadRowID] = X[threadColId * numCol + threadRowID] * Y[threadColId * numCol + threadRowID];
  // if (threadRowID < numRow && threadColId < numCol) {
  Z[threadRowID * numCol + threadColId] = X[threadRowID * numCol + threadColId] * Y[threadRowID * numCol + threadColId];
  // }

}



int main()
{
  const size_t ARRAY_DIM = 2048; // 4096;
  const size_t ARRAY_BYTES = ARRAY_DIM * ARRAY_DIM * sizeof(float);
  size_t NUM_EXEC = 30;

  // Array Initialization
  float* X;
  float* Y;
  float* Z;

  // Initialize C implementation of Hadamard Product, reference for error checking
  float* C;
  C = (float*)malloc(ARRAY_DIM * ARRAY_DIM * sizeof(float));

  // Array Malloc
  cudaMallocManaged(&X, ARRAY_BYTES);
  cudaMallocManaged(&Y, ARRAY_BYTES);
  cudaMallocManaged(&Z, ARRAY_BYTES);

  // get gpu ID
  int device = -1;
  cudaGetDevice(&device);

  // Mem advise
  cudaMemAdvise(X, ARRAY_BYTES, cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);
  cudaMemAdvise(X, ARRAY_BYTES, cudaMemAdviseSetReadMostly, cudaCpuDeviceId);
  cudaMemAdvise(Y, ARRAY_BYTES, cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);
  cudaMemAdvise(Y, ARRAY_BYTES, cudaMemAdviseSetReadMostly, cudaCpuDeviceId);

  // "prefetch data" to create CPU page memory
  cudaMemPrefetchAsync(X, ARRAY_BYTES, cudaCpuDeviceId, NULL);
  // "prefetch data" to create CPU page memory
  cudaMemPrefetchAsync(Y, ARRAY_BYTES, cudaCpuDeviceId, NULL);
  // "prefetch data" to create GPU page memory
  cudaMemPrefetchAsync(Z, ARRAY_BYTES, device, NULL);

  // initialize array contents
  for (size_t i = 0; i < ARRAY_DIM; ++i) {
    for (size_t j = 0; j < ARRAY_DIM; ++j) {
      X[ARRAY_DIM * i + j] = 1;
      Y[ARRAY_DIM * i + j] = 1;
    }
  }

  // "Prefetch data" from CPU-GPU
  cudaMemPrefetchAsync(X, ARRAY_BYTES, device, NULL);
  cudaMemPrefetchAsync(Y, ARRAY_BYTES, device, NULL);

  C_Hadamard(ARRAY_DIM, ARRAY_DIM, C, X, Y);

  // setup CUDA kernel
  // https://www.cs.emory.edu/~cheung/Courses/355/Syllabus/94-CUDA/2D-grids.html
  size_t threadDimBlockx = 16;
  size_t threadDimBlocky = 16;

  dim3 blockShape = dim3(threadDimBlockx, threadDimBlocky);
  // https://selkie.macalester.edu/csinparallel/modules/GPUProgramming/build/html/CUDA2D/CUDA2D.html
  dim3 gridShape = dim3(ARRAY_DIM/threadDimBlockx, ARRAY_DIM/threadDimBlocky);

  for (size_t i = 0; i < NUM_EXEC; ++i) {
    cuda_hadamard <<< gridShape, blockShape >>> (ARRAY_DIM, ARRAY_DIM, Z, X, Y);
  }

  cudaDeviceSynchronize();

  // "Prefetch data" from GPU-CPU
  cudaMemPrefetchAsync(X, ARRAY_BYTES, cudaCpuDeviceId, NULL);
  cudaMemPrefetchAsync(Y, ARRAY_BYTES, cudaCpuDeviceId, NULL);
  cudaMemPrefetchAsync(Z, ARRAY_BYTES, cudaCpuDeviceId, NULL);


  // dump_arr(ARRAY_DIM, ARRAY_DIM, X);
  // printf("\n\n");
  // dump_arr(ARRAY_DIM, ARRAY_DIM, Z);

  // error checking
  size_t errCount = 0;

  for (size_t i = 0; i < ARRAY_DIM; ++i) {
    for (size_t j = 0; j <ARRAY_DIM; ++j ) {
      if (C[i * ARRAY_DIM + j] != Z[i * ARRAY_DIM + j]) {
        errCount++;
      }
    }
  }

  printf("Total error count: %lu", errCount);

  cudaFree(X);
  cudaFree(Y);
  cudaFree(Z);


  return 0;
}


Overwriting hadamard_cuda16x16.cu


In [38]:
%%shell
nvcc -o hadamard_cuda16x16 hadamard_cuda16x16.cu -arch=sm_75

In [39]:
%%shell
nvprof ./hadamard_cuda16x16

==11567== NVPROF is profiling process 11567, command: ./hadamard_cuda16x16
Total error count: 0==11567== Profiling application: ./hadamard_cuda16x16
==11567== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  17.801ms        30  593.36us  591.96us  594.65us  cuda_hadamard(unsigned long, unsigned long, float*, float*, float*)
      API calls:   87.09%  223.39ms         3  74.464ms  44.159us  223.29ms  cudaMallocManaged
                    6.91%  17.719ms         1  17.719ms  17.719ms  17.719ms  cudaDeviceSynchronize
                    4.98%  12.774ms         8  1.5968ms  14.694us  3.9716ms  cudaMemPrefetchAsync
                    0.82%  2.1044ms         3  701.46us  566.70us  827.91us  cudaFree
                    0.12%  298.26us        30  9.9420us  3.7800us  174.56us  cudaLaunchKernel
                    0.07%  187.79us       114  1.6470us     120ns  93.753us  cuDeviceGetAttribute
                    0.01% 

## CUDA with Shared Memory

In [50]:
%%writefile hadamard_cuda_shared16x16.cu
#include <cstddef>
#include <cstdio>
#include <stdlib.h>
#include <stdio.h>
#include <cuda_runtime.h>


void dump_arr(size_t numCol, size_t numRow, float* Arr)
{
    for (size_t j = 0; j < numCol; ++j) {
      for (size_t i = 0; i < numRow; ++i) {
      printf("%.2f ", Arr[i * numCol + j]);
    }
    printf("\n");
  }
}

__global__
void cuda_hadamard(size_t numCol, size_t numRow, float* Z, float* X, float* Y)
{

  size_t threadRowID = blockIdx.x * blockDim.x + threadIdx.x;
  size_t threadColId = blockIdx.y * blockDim.y + threadIdx.y;
  __shared__ float shData[1];

  shData[0] = X[threadRowID * numCol + threadColId] * Y[threadRowID * numCol + threadColId];

  __syncthreads();

  Z[threadRowID * numCol + threadColId] = shData[0]; // shared maemory

  // Z[threadRowID * numCol + threadColId] = X[threadRowID * numCol + threadColId] * Y[threadRowID * numCol + threadColId];
}



int main()
{
  const size_t ARRAY_DIM =   4096; //1024, 2048, 4096
  const size_t ARRAY_BYTES = ARRAY_DIM * ARRAY_DIM * sizeof(float);
  size_t NUM_EXEC = 30;

  // Array Initialization
  float* X;
  float* Y;
  float* Z;

  // Initialize C implementation of Hadamard Product, reference for error checking
  // float* C;
  // C = (float*)malloc(ARRAY_DIM * ARRAY_DIM * sizeof(float));

  // Array Malloc
  cudaMallocManaged(&X, ARRAY_BYTES);
  cudaMallocManaged(&Y, ARRAY_BYTES);
  cudaMallocManaged(&Z, ARRAY_BYTES);

  // get gpu ID
  int device = -1;
  cudaGetDevice(&device);

  // Mem advise
  cudaMemAdvise(X, ARRAY_BYTES, cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);
  cudaMemAdvise(X, ARRAY_BYTES, cudaMemAdviseSetReadMostly, cudaCpuDeviceId);
  cudaMemAdvise(Y, ARRAY_BYTES, cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);
  cudaMemAdvise(Y, ARRAY_BYTES, cudaMemAdviseSetReadMostly, cudaCpuDeviceId);

  // "prefetch data" to create CPU page memory
  cudaMemPrefetchAsync(X, ARRAY_BYTES, cudaCpuDeviceId, NULL);
  // "prefetch data" to create CPU page memory
  cudaMemPrefetchAsync(Y, ARRAY_BYTES, cudaCpuDeviceId, NULL);
  // "prefetch data" to create GPU page memory
  cudaMemPrefetchAsync(Z, ARRAY_BYTES, device, NULL);

  // initialize array contents
  for (size_t i = 0; i < ARRAY_DIM; ++i) {
    for (size_t j = 0; j < ARRAY_DIM; ++j) {
      X[ARRAY_DIM * i + j] = 1;
      Y[ARRAY_DIM * i + j] = 1;
    }
  }

  // "Prefetch data" from CPU-GPU
  cudaMemPrefetchAsync(X, ARRAY_BYTES, device, NULL);
  cudaMemPrefetchAsync(Y, ARRAY_BYTES, device, NULL);

  // setup CUDA kernel
  // https://www.cs.emory.edu/~cheung/Courses/355/Syllabus/94-CUDA/2D-grids.html
  size_t threadDimBlockx = 32; // 8, 16, 32
  size_t threadDimBlocky = 32; // 8, 16, 32

  dim3 blockShape = dim3(threadDimBlockx, threadDimBlocky);
  // https://selkie.macalester.edu/csinparallel/modules/GPUProgramming/build/html/CUDA2D/CUDA2D.html
  // https://medium.com/@harsh20111997/cuda-programming-2d-convolution-8476300f566e
  dim3 gridShape = dim3( (ARRAY_DIM + threadDimBlockx - 1) / threadDimBlockx, (ARRAY_DIM + threadDimBlocky - 1)/threadDimBlocky );

  for (size_t i = 0; i < NUM_EXEC; ++i) {
    cuda_hadamard <<< gridShape, blockShape >>> (ARRAY_DIM, ARRAY_DIM, Z, X, Y);
  }

  cudaDeviceSynchronize();

  // "Prefetch data" from GPU-CPU
  cudaMemPrefetchAsync(X, ARRAY_BYTES, cudaCpuDeviceId, NULL);
  cudaMemPrefetchAsync(Y, ARRAY_BYTES, cudaCpuDeviceId, NULL);
  cudaMemPrefetchAsync(Z, ARRAY_BYTES, cudaCpuDeviceId, NULL);


  // error checking
  size_t errCount = 0;

  for (size_t i = 0; i < ARRAY_DIM; ++i) {
    for (size_t j = 0; j <ARRAY_DIM; ++j ) {
      if (X[i * ARRAY_DIM + j] * Y[i * ARRAY_DIM + j] != Z[i * ARRAY_DIM + j]) {
        errCount++;
      }
    }
  }

  printf("Total error count: %lu", errCount);

  cudaFree(X);
  cudaFree(Y);
  cudaFree(Z);


  return 0;
}


Overwriting hadamard_cuda_shared16x16.cu


In [53]:
%%shell
nvcc -o hadamard_cuda_shared16x16 hadamard_cuda_shared16x16.cu -arch=sm_75

In [55]:
%%shell
nvprof ./hadamard_cuda_shared16x16

==18358== NVPROF is profiling process 18358, command: ./hadamard_cuda_shared16x16
Total error count: 0==18358== Profiling application: ./hadamard_cuda_shared16x16
==18358== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  133.26ms        30  4.4421ms  4.2487ms  4.5832ms  cuda_hadamard(unsigned long, unsigned long, float*, float*, float*)
      API calls:   52.78%  213.56ms         3  71.187ms  27.666us  213.48ms  cudaMallocManaged
                   32.92%  133.18ms         1  133.18ms  133.18ms  133.18ms  cudaDeviceSynchronize
                   12.13%  49.059ms         8  6.1324ms  39.872us  15.974ms  cudaMemPrefetchAsync
                    2.05%  8.2953ms         3  2.7651ms  2.3020ms  3.2580ms  cudaFree
                    0.07%  296.74us        30  9.8910us  3.4590us  181.76us  cudaLaunchKernel
                    0.04%  165.03us       114  1.4470us     113ns  63.037us  cuDeviceGetAttribute
            